# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import pickle

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.pipeline import Pipeline
# from pipelinehelper import PipelineHelper
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

random_state = 0

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# load data from database
engine = create_engine('sqlite:///disastermanagement.db')
df = pd.read_sql_table('labeledmessages', engine)
X = df.message
y = df[df.columns[4:]]

In [5]:
df.head()

,id,message,original,genre,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [7]:
y.head()

,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)","('search_and_rescue',)","('security',)","('military',)","('child_alone',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    """
    Method to process the text data into lemmatized tokens

    Args:
        text: text data to be processed

    Returns:
        list: clean_tokes list with tokens extracted from the processed text data 
    """
    # normalize case and remove leading/trailing white space and punctuation
    text = re.sub("\W"," ", text.lower().strip())
    
    # tokenize
    tokens = word_tokenize(text)
    
    # initiate stopword
    stop_words = stopwords.words("english")
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token to
    # lemmatize and remove stopwords  
    clean_tokens = []
    
    for tok in tokens:
        if tok not in stop_words:

            clean_tok = lemmatizer.lemmatize(tok)
            clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
#spliting data into test and train
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [11]:
# training th pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_pred = pipeline.predict(X_test)

In [13]:
for index, column in enumerate(y_test):
    print(column, classification_report(y_test[column], y_pred[:, index]))

('related',)              precision    recall  f1-score   support

          0       0.63      0.46      0.54      1551
          1       0.84      0.91      0.88      4965
          2       0.30      0.37      0.33        38

avg / total       0.79      0.80      0.79      6554

('request',)              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5434
          1       0.81      0.44      0.57      1120

avg / total       0.88      0.89      0.87      6554

('offer',)              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

('aid_related',)              precision    recall  f1-score   support

          0       0.75      0.86      0.80      3806
          1       0.75      0.59      0.66      2748

avg / total       0.75      0.75      0.74      6554

('medical_help',)              pr

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f643e47ad90>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [15]:
parameters = {
    'clf__estimator__n_estimators': [50, 100],
    #'clf__estimator__min_samples_split': [2, 3, 4],
   
    
}

# GridSearchCV help: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
cv = GridSearchCV(pipeline, parameters, verbose=10, n_jobs=-1)

In [16]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=100 ................................
[CV] clf__estimator__n_estimators=100 ................................
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=50, score=0.26365578272810497, total=18.6min
[CV]  clf__estimator__n_estimators=50, score=0.25190723222459566, total=18.7min


[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed: 20.9min remaining: 41.7min


[CV]  clf__estimator__n_estimators=50, score=0.25190723222459566, total=18.9min


[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed: 21.0min remaining: 21.0min


[CV]  clf__estimator__n_estimators=100, score=0.26350320415013734, total=27.4min


[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed: 29.0min remaining: 14.5min


[CV]  clf__estimator__n_estimators=100, score=0.2581629539212695, total=27.5min
[CV]  clf__estimator__n_estimators=100, score=0.25968873970094597, total=27.6min


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 29.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 29.2min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [17]:
cv.best_params_

{'clf__estimator__n_estimators': 100}

In [18]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
model_tuned = cv.best_estimator_
new_pred = model_tuned.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], new_pred[:,i]))

('related',) :
             precision    recall  f1-score   support

          0       0.73      0.44      0.55      1551
          1       0.84      0.95      0.89      4965
          2       0.30      0.42      0.35        38

avg / total       0.81      0.82      0.81      6554

('request',) :
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5434
          1       0.85      0.49      0.62      1120

avg / total       0.89      0.90      0.89      6554

('offer',) :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

('aid_related',) :
             precision    recall  f1-score   support

          0       0.80      0.85      0.82      3806
          1       0.77      0.70      0.74      2748

avg / total       0.79      0.79      0.79      6554

('medical_help',) :
     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
accuracy = (new_pred == y_test).mean()
accuracy

('related',)                   0.822399
('request',)                   0.898230
('offer',)                     0.996033
('aid_related',)               0.787458
('medical_help',)              0.920812
('medical_products',)          0.953311
('search_and_rescue',)         0.974825
('security',)                  0.982911
('military',)                  0.970857
('child_alone',)               1.000000
('water',)                     0.954684
('food',)                      0.941715
('shelter',)                   0.934696
('clothing',)                  0.984590
('money',)                     0.975435
('missing_people',)            0.989625
('refugees',)                  0.967653
('death',)                     0.959414
('other_aid',)                 0.867409
('infrastructure_related',)    0.930882
('transport',)                 0.954837
('buildings',)                 0.951327
('electricity',)               0.980317
('tools',)                     0.992676
('hospitals',)                 0.989319


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
tfIdf_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 100)))
    ])
# AdaBoost Classifer help: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
tfIdf_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ator=None,
          learning_rate=1.0, n_estimators=100, random_state=None),
           n_jobs=1))])

In [23]:
y_pred_tfIdf = tfIdf_pipeline.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred_tfIdf[:,i]))

('related',) :
             precision    recall  f1-score   support

          0       0.66      0.17      0.27      1513
          1       0.79      0.97      0.87      4991
          2       0.38      0.20      0.26        50

avg / total       0.76      0.78      0.73      6554

('request',) :
             precision    recall  f1-score   support

          0       0.91      0.96      0.94      5434
          1       0.76      0.53      0.63      1120

avg / total       0.88      0.89      0.88      6554

('offer',) :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6532
          1       0.00      0.00      0.00        22

avg / total       0.99      0.99      0.99      6554

('aid_related',) :
             precision    recall  f1-score   support

          0       0.77      0.87      0.82      3816
          1       0.78      0.63      0.70      2738

avg / total       0.77      0.77      0.77      6554

('medical_help',) :
     

### 9. Export your model as a pickle file

In [24]:
pickle.dump(model_tuned, open('message_lr_classifier.pkl','wb'))

In [25]:
model_tuned.score(X_test, y_test)

0.81400671345743059

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
pickle.dump(model_tuned, open('message_lr_classifier.pkl','wb'))